In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


In [3]:
!nvidia-smi

Wed Mar 25 14:06:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:37:00.0 Off |                  Off |
| 30%   45C    P2    37W / 250W |   5012MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
import sys
sys.path.append('..')

In [5]:
from bert.model import create_albert_model
from bert.losses import ECE, masked_sparse_categorical_crossentropy
from bert.optimization import create_optimizer

opt = create_optimizer(1E-4, 500, 10000)

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")

with strategy.scope():
    model = create_albert_model(model_dimension=512,
                                transformer_dimension=512 * 4,
                                num_attention_heads=512 // 64,
                                num_transformer_layers=6,
                                vocab_size=24,
                                dropout_rate=0.,
                                max_relative_position=128)
    
    model.compile(
        loss=masked_sparse_categorical_crossentropy,
        metrics=[ECE],
        optimizer=opt,
        experimental_run_tf_function=True)    

    
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 512)         12288     
_________________________________________________________________
transformer (Transformer)    (None, None, 512)         3168832   
_________________________________________________________________
transformer_1 (Transformer)  (None, None, 512)         3168832   
_________________________________________________________________
transformer_2 (Transformer)  (None, None, 512)         3168832   
_________________________________________________________________
transformer_3 (Transformer)  (None, None, 512)         3168832   
_________________________________________________________________
transformer_4 (Transformer)  (None, None, 512)         316883

In [8]:
from bert.dataset import create_masked_input_dataset

with tf.device('/CPU:0'):
    training_data = create_masked_input_dataset(
        sequence_path='../../uniparc_data/train_uniref100.txt.gz',
        max_sequence_length=1024,
        fix_sequence_length=True,
        batch_size=2)

    training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

    valid_data = create_masked_input_dataset(
        sequence_path='../../uniparc_data/dev_uniref50.txt.gz',
        max_sequence_length=1024,
        batch_size=2)

    valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
model.fit(training_data, steps_per_epoch=500, epochs=5,
          verbose=1, validation_data=valid_data, validation_steps=10)

Train for 500 steps, validate for 10 steps
Epoch 1/5
500/500 [==============================] - 94s 187ms/step - loss: 2.9139 - ECE: 18.5889 - val_loss: 2.7454 - val_ECE: 15.6050
Epoch 2/5
500/500 [==============================] - 85s 169ms/step - loss: 2.7084 - ECE: 15.0838 - val_loss: 2.7179 - val_ECE: 15.2255
Epoch 3/5
500/500 [==============================] - 85s 170ms/step - loss: 2.6956 - ECE: 14.8992 - val_loss: 2.7275 - val_ECE: 15.3837
Epoch 4/5
500/500 [==============================] - 85s 169ms/step - loss: 2.6910 - ECE: 14.8331 - val_loss: 2.6737 - val_ECE: 14.5106
Epoch 5/5
500/500 [==============================] - 85s 170ms/step - loss: 2.6911 - ECE: 14.8383 - val_loss: 2.6791 - val_ECE: 14.6144


In [ ]:
masked, true = next(iter(training_data))
predictions = model.predict(masked)

In [ ]:
masked[:, 0]

In [ ]:
model.save('/scratch/pstjohn/uniparc_checkpoints/test.h5')